In [1]:
# !export CUDA_LAUNCH_BLOCKING=1
# !export TORCH_USE_CUDA_DSA=true
# !export TOKENIZERS_PARALLELISM=false

In [2]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["WANDB_DISABLED"] = "true"

In [3]:
!nvidia-smi

Tue May 30 22:07:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:21:01.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 32768MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# <your imports>
import numpy as np
import torch
from torch.optim import Adam
import torch.nn as nn
from tqdm import tqdm
from omegaconf import OmegaConf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from transformers import TrainingArguments, Trainer
from transformers import BertForSequenceClassification, XLMRobertaXLConfig
from transformers import EarlyStoppingCallback

import sys
sys.path.append('../src')
from src.preprocessing import Preprocessing
from matplotlib import pyplot as plt

# models
from src.models import BertClassifier

2023-05-30 22:07:30.779357: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# ignore fucking warnings
import warnings
warnings.filterwarnings("ignore")
from transformers import logging
logging.set_verbosity_warning()

In [6]:
def custom_f1(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    # f1_scores = f1_score(y_true=labels, y_pred=pred, average="weighted")
    
    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = (f1_scores[2] + f1_scores[2]) / 2
    return {
        "accuracy": accuracy,
        "f1_score for negative": f1_scores[0],
        "f1_score for neutral": f1_scores[1],
        "f1_score for positive": f1_scores[2],
        # "f1_score": final_score
    }

def compute_metrics(p):
    pred, labels = p
    # print(pred, labels)
    # print(np.unique(labels, return_counts=True), labels.shape)
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [7]:
preprocessor = Preprocessing()

# result = preprocessor.get_dataloaders()
result = preprocessor.get_datasets()

train_set = result["train_set"]
val_set = result["val_set"]
# plt.hist(val_set.labels, bins=3)
None

In [8]:
# train_set[1]

In [9]:
conf = OmegaConf.load("config.yaml").general
for i in conf.keys():
    print(f'{i}: {conf[i]}')

dataset_name: ruSentNE_lemmatized
batch_size: 16
device: cpu
dataloader_shuffle: True
pretrained_model: bert-base-uncased
num_classes: 3
num_workers: 1
lr: 1e-06
epochs: 2
eps: 1e-08
optimizer: AdamW
eval_steps: 10
logging_steps: 10


In [10]:
# help(AutoModelForSequenceClassification)

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    # "cardiffnlp/twitter-roberta-base-sentiment",
    "coderSounak/finetuned_twitter_sentiment_LSTM", 
    # 'cardiffnlp/twitter-xlm-roberta-base-sentiment',
    num_labels=3,
)
model.cuda()
None

In [12]:
# model

In [13]:
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=conf.eval_steps,
    per_device_train_batch_size=conf.batch_size,
    per_device_eval_batch_size=conf.batch_size,
    num_train_epochs=conf.epochs,
    seed=69,
    logging_steps=conf.logging_steps,
    # learning_rate=conf.lr,
    learning_rate=3e-7,
    load_best_model_at_end=True,
    # no_cuda=True
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [12]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_set,
    eval_dataset=val_set,
    # compute_metrics=compute_metrics,
    compute_metrics=custom_f1,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    # num_workers=1,
)

## Train loop

In [13]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,F1 Score for negative,F1 Score for neutral,F1 Score for positive
10,1.107500,1.102365,0.363158,0.495706,0.079681,0.281938
20,1.125100,1.102294,0.363158,0.496314,0.079681,0.281319
30,1.101200,1.102098,0.360526,0.491965,0.080000,0.281996
40,1.103200,1.102000,0.363158,0.495098,0.080972,0.280088
50,1.093700,1.101853,0.361842,0.492005,0.080972,0.282609
60,1.117200,1.101590,0.364474,0.496314,0.081633,0.281996
70,1.095500,1.101311,0.361842,0.493827,0.073770,0.283262
80,1.131600,1.101100,0.363158,0.490613,0.065844,0.301255
90,1.106700,1.101009,0.363158,0.489335,0.058091,0.307054
100,1.114800,1.101001,0.364474,0.491228,0.058333,0.307054


TrainOutput(global_step=380, training_loss=1.1054636905067845, metrics={'train_runtime': 32.2268, 'train_samples_per_second': 188.601, 'train_steps_per_second': 11.791, 'total_flos': 21479593651200.0, 'train_loss': 1.1054636905067845, 'epoch': 2.0})

## Test

In [22]:
import pandas as pd

In [14]:
preprocessor = Preprocessing()
result = preprocessor.get_dataloaders()
train_dl = result["train_dl"]
val_dl = result["val_dl"]

In [26]:
# def test_loader(string):

filaname = "final_data.csv"

with open("config.yaml") as f:
self.full_config = yaml.load(f, Loader=yaml.FullLoader)
self.config = self.full_config["preprocessing"]

df = pd.read_csv(f"data/{filaname}", '\t')
display(df.head())

if self.config["tokenizer"] == "BertTokenizerFast":
    self.tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
elif self.config["tokenizer"] == "RobertaTokenizerFast":
    self.tokenizer = RobertaTokenizerFast.from_pretrained(
        "blinoff/roberta-base-russian-v0", max_len=512
    )
elif self.config["tokenizer"] == "RobertaTokenizer":
    self.tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
elif self.config["tokenizer"] == "GPT2Tokenizer":
    self.tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    self.tokenizer.add_special_tokens({"pad_token": "[PAD]"})
elif self.config["tokenizer"] == "DistilBertTokenizer":
    self.tokenizer = DistilBertTokenizer.from_pretrained(
        "distilbert-base-uncased"
    )
else:
    raise ValueError(f"Tokenizer {self.config['tokenizer']} not supported")

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel
0,Абдул Реза Шайхулислами выразил готовность Ира...,Абдул Реза Шайхулислами,PERSON,0,23
1,Абдул Реза Шайхулислами выразил готовность Ира...,афганцев,NATIONALITY,157,165
2,Абдул Реза Шайхулислами выразил готовность Ира...,правительству Афганистана,ORGANIZATION,68,93
3,В своем выступлении Абдул Реза Шайхулислами от...,Абдул Реза Шайхулислами,PERSON,20,43
4,"В свою очередь, Зорар Ахмад Мокбел заявил о то...",Ирана,COUNTRY,86,91


In [27]:
with torch.no_grad():
    outputs = []
    for batch in train_dl:
        inputs = {
            'input_ids': batch['input_ids'].cuda(),
            'attention_mask': batch['attention_mask'].cuda(),
            'labels': batch['label'].cuda(),
        }
        output = model(
            **inputs
        )
        # print(output.logits.cpu())
        outputs.append(output.logits.cpu())
outputs = np.vstack(outputs)
outputs.shape

(3039, 3)

## Custom trainer

In [13]:
from trainer import CustomTrainer

In [14]:
preprocessor = Preprocessing()

result = preprocessor.get_dataloaders()

train_dl = result["train_dl"]
val_dl = result["val_dl"]

In [19]:
for i in val_dl:
    # print(i)
    i['input_ids'].double().to("cuda:0")
    break

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [16]:
trainer = CustomTrainer(model)

Initialized training config with params: {'dataset_name': 'semEval', 'batch_size': 16, 'device': 'cpu', 'dataloader_shuffle': True, 'pretrained_model': 'bert-base-uncased', 'num_classes': 3, 'num_workers': 1, 'lr': 1e-06, 'epochs': 5, 'eps': 1e-08, 'optimizer': 'AdamW', 'eval_steps': 10, 'logging_steps': 10}


In [17]:
trainer.train(train_dl, val_dl)

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/48 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

## Evaluation/Weighting metric

In [14]:
import optuna

In [15]:
def make_weighted_metric(p):
    print(p)
    global weights
    pred, labels = p
    if weights is None:
        weights = np.ones(pred.shape[1])
    pred = np.argmax(pred*weights, axis=1)

    f1_scores = f1_score(y_true=labels, y_pred=pred, average=None)
    final_score = np.mean([f1_scores[0], f1_scores[2]])
    # return {"custom F1 score": final_score}
    # return {
    #     "F1 for class 0": f1_scores[0],
    #     "F1 for class 1": f1_scores[1],
    #     "F1 for class 2": f1_scores[2]
    # }
    # f1_scores(f1_scores)
    # print(f1_scores)
    return {"f1_score": final_score}

In [16]:
def target_func(weight_1, weight_2, weight_3):
    global weights
    weights = np.array([weight_1, weight_2, weight_3])
    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=train_set,
        eval_dataset=val_set,
        # compute_metrics=compute_metrics,
        compute_metrics=make_weighted_metric,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )
    result = trainer.evaluate()
    # final_result = (result['eval_F1 for class 0'] + result['eval_F1 for class 2']) / 3
    # return final_result
    # print(result)
    return result['eval_f1_score']

In [17]:
def objective(trial):
    weight_1 = trial.suggest_float("weight_1", 0, 1)
    weight_2 = trial.suggest_float("weight_2", 0, 1)
    weight_3 = trial.suggest_float("weight_3", 0, 1)
    
    result = target_func(weight_1, weight_2, weight_3)
    
    return result

In [18]:
study = optuna.create_study(directions=["maximize"])
study.optimize(objective, n_trials=300, timeout=300)

[I 2023-05-30 04:14:19,274] A new study created in memory with name: no-name-e4f8a9ac-402a-45a3-b308-a1117f85e175


[I 2023-05-30 04:14:19,852] Trial 0 finished with value: 0.4034500800276295 and parameters: {'weight_1': 0.5874864400324022, 'weight_2': 0.7949162975558586, 'weight_3': 0.39583995499501257}. Best is trial 0 with value: 0.4034500800276295.


[I 2023-05-30 04:14:20,427] Trial 1 finished with value: 0.37269644699985255 and parameters: {'weight_1': 0.21986591300904557, 'weight_2': 0.7659683455244781, 'weight_3': 0.6905492279043823}. Best is trial 0 with value: 0.4034500800276295.


[I 2023-05-30 04:14:21,006] Trial 2 finished with value: 0.36189943476212905 and parameters: {'weight_1': 0.22831453534242252, 'weight_2': 0.8193081086458855, 'weight_3': 0.9886644792971195}. Best is trial 0 with value: 0.4034500800276295.


[I 2023-05-30 04:14:21,597] Trial 3 finished with value: 0.4037056036080877 and parameters: {'weight_1': 0.7837839098415998, 'weight_2': 0.3401212219545309, 'weight_3': 0.5508885739160266}. Best is trial 3 with value: 0.4037056036080877.


...

[I 2023-05-30 04:14:31,125] Trial 19 finished with value: 0.39930377882185114 and parameters: {'weight_1': 0.41462483936635586, 'weight_2': 0.6958596356120349, 'weight_3': 0.2526502886144946}. Best is trial 8 with value: 0.43599866765939105.


[I 2023-05-30 04:14:31,722] Trial 20 finished with value: 0.42031636208419776 and parameters: {'weight_1': 0.8501997805701872, 'weight_2': 0.5593213766190698, 'weight_3': 0.009497657839717413}. Best is trial 8 with value: 0.43599866765939105.


[I 2023-05-30 04:14:32,321] Trial 21 finished with value: 0.42926220595772396 and parameters: {'weight_1': 0.8634216313936902, 'weight_2': 0.5037376361456465, 'weight_3': 0.14971883476648978}. Best is trial 8 with value: 0.43599866765939105.


[W 2023-05-30 04:14:32,591] Trial 22 failed with parameters: {'weight_1': 0.6933186897110589, 'weight_2': 0.3534082922096264, 'weight_3': 0.15444726288935004} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/lazarev/disk/lazarev/miniconda3/envs/study/lib/python3.10/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_21255/2436653937.py", line 6, in objective
    result = target_func(weight_1, weight_2, weight_3)
  File "/tmp/ipykernel_21255/1414780235.py", line 13, in target_func
    result = trainer.evaluate()
  File "/home/lazarev/disk/lazarev/miniconda3/envs/study/lib/python3.10/site-packages/transformers/trainer.py", line 2993, in evaluate
    output = eval_loop(
  File "/home/lazarev/disk/lazarev/miniconda3/envs/study/lib/python3.10/site-packages/transformers/trainer.py", line 3174, in evaluation_loop
    loss, logits, labels = self.prediction_step(model, inp

KeyboardInterrupt: 

In [18]:
study.best_params, study.best_value

NameError: name 'study' is not defined